# AI for cell recognition

To help physicians we need to develop an AI that can classify cells into different types. For that, we have opened huge slide images, detected many cells, and extracted a feature vector for each of them.

We also have asked the experts to annotate, for each cell class and many examples, if the cell belongs or not to the given class. Our data is organized as follows:
- `annotations_{class}.csv` are CSVs for the three classes of interest, namely `(lymphocyte, lymphoplasmocyte, plasmocyte)`. The first column represents the cell index and the second a binary value (1 if the cell belongs to the class, else 0). 
- `cell_dataset.pkl` is the file containing the features and bounding boxes for the cells. We will not use the bounding boxes for now, they will be used later for visualization purposes. 

We now load the data and print some things:

In [13]:
import pickle
import csv

# load features
with open('cell_dataset.pkl', 'rb') as f:
    features = pickle.load(f)['feats']
print('features is a matrix of dimension (N,D)=', features.shape)

# load annotations for all classes
classes = ("lymphocyte", "lymphoplasmocyte", "plasmocyte")
annotations = {c: ([], []) for c in classes}  # init empty dict per class
for i, c in enumerate(classes):
    print("-"*80)
    print('class', i, 'is', c)
    annotation_file = f"annotations_{c}.csv"
    # load annotation file
    with open(annotation_file, 'r') as f:
        reader = csv.reader(f)
        # skip header
        for row in reader:
            annotations[c][0].append(int(row[0]))  # cell index
            annotations[c][1].append(bool(int(row[1])))  # is positive?
    print("number of annotations for class", c, "is", len(annotations[c][0]))
    print("number of positive annotations for class", c, "is", sum(annotations[c][1]))
         

features is a matrix of dimension (N,D)= (984007, 384)
--------------------------------------------------------------------------------
class 0 is lymphocyte
number of annotations for class lymphocyte is 551
number of positive annotations for class lymphocyte is 255
--------------------------------------------------------------------------------
class 1 is lymphoplasmocyte
number of annotations for class lymphoplasmocyte is 468
number of positive annotations for class lymphoplasmocyte is 138
--------------------------------------------------------------------------------
class 2 is plasmocyte
number of annotations for class plasmocyte is 484
number of positive annotations for class plasmocyte is 134


## Reformatting the data
We can see that, contrary to how most multi-class classification problems are organized, our annotations are binary for each class. However, we would like to use common techniques on multi-class classification, so it would be nice to convert our dataset to the standard format.

What is this format? Well, it is defined by two variables `X` and `y`. The matrix `X` is of shape `(A,D)`, where `A` is the total number of annotations and `D` the feature dimension. `X` contains real values. The vector `y` has shape `D`, and contains integers, one representing each class. Note that we have a total of 4 classes `(0,1,2,3)`, because we have the three cell types + the class of cells that are not any of these three types.

### Task 1: Consistency
We need to check that the data is of good quality. For that, check that the annotations are consistent: this is, check that all the annotations refer to different indices. If there are annotations for the same index, then the annotation should not be contradictory. Each annotation should belong to only one class.

If there are any indices that were inconsistently annotated, print them out. 

### Task 2: Formatting
Now that we have ensured consistency, we need to format the labels or annotations in variables `X,y` as described above. For that, create a list of annotated indices, the list of corresponding labels (0,1,2, or 3) and the list of features corresponding to the indices.

In [14]:
# Initialiser un dictionnaire pour stocker les annotations de toutes les classes par index
combined_annotations = {}

# Liste pour stocker toutes les incohérences
inconsistent_indices = []

# Fusionner les annotations pour chaque classe
for c in classes:
    indices = annotations[c][0]
    labels = annotations[c][1]
    for idx, label in zip(indices, labels):
        if idx not in combined_annotations:
            combined_annotations[idx] = {c: label}
        else:
            if c in combined_annotations[idx] and combined_annotations[idx][c] != label:
                # Ajouter l'incohérence au tableau des incohérences
                inconsistent_indices.append({
                    'index': idx,
                    'class': c,
                    'existing_label': combined_annotations[idx][c],
                    'new_label': label,
                    'type': 'incoherent'
                })
        combined_annotations[idx][c] = label

# Détecter les indices ayant plusieurs classes
for idx, class_labels in combined_annotations.items():
    # Vérifier si l'index appartient à plus d'une classe (labels True pour plusieurs classes)
    positive_classes = [cls for cls, label in class_labels.items() if label]
    if len(positive_classes) > 1:
        # Ajouter l'indice avec plusieurs classes au tableau des incohérences
        inconsistent_indices.append({
            'index': idx,
            'classes': positive_classes,
            'type': 'multiple_classes'
        })

# Afficher les incohérences
print("Indices incohérents ou ayant plusieurs classes :")
for inconsistency in inconsistent_indices:
    if inconsistency['type'] == 'incoherent':
        print(f"Index {inconsistency['index']} a une incohérence entre les classes {inconsistency['class']} : "
              f"ancienne étiquette = {inconsistency['existing_label']}, nouvelle étiquette = {inconsistency['new_label']}")
    elif inconsistency['type'] == 'multiple_classes':
        print(f"Index {inconsistency['index']} appartient aux classes : {inconsistency['classes']}")

# Résumé
if not inconsistent_indices:
    print("Aucune incohérence détectée.")
else:
    print(f"\nNombre total d'incohérences ou d'indices ayant plusieurs classes : {len(inconsistent_indices)}")


Indices incohérents ou ayant plusieurs classes :
Index 585154 a une incohérence entre les classes lymphocyte : ancienne étiquette = False, nouvelle étiquette = True
Index 732406 a une incohérence entre les classes lymphoplasmocyte : ancienne étiquette = True, nouvelle étiquette = False
Index 968583 a une incohérence entre les classes plasmocyte : ancienne étiquette = True, nouvelle étiquette = False
Index 729897 appartient aux classes : ['lymphoplasmocyte', 'plasmocyte']

Nombre total d'incohérences ou d'indices ayant plusieurs classes : 4


In [5]:
annotations[classes[2]][0]
729897 in annotations[classes[2]][0]

True

In [15]:
import numpy as np
import pickle

# On suppose que les incohérences ont déjà été détectées et stockées dans la variable `incoherences`
incoherent_indices = [incoherence['index'] for incoherence in inconsistent_indices]

# Charger les données existantes (cell_dataset.pkl)
with open('cell_dataset.pkl', 'rb') as f:
    data = pickle.load(f)

# Supposons que 'data['feats']' est un tableau numpy des caractéristiques des cellules
# Si 'data['feats']' est un tableau numpy, les indices peuvent être simplement les indices de ligne.
cell_indices = np.arange(len(data['feats']))  # Création d'une liste d'indices basée sur la longueur de 'feats'

# Filtrer les cellules en excluant celles qui ont des incohérences
filtered_feats = [feats for idx, feats in zip(cell_indices, data['feats']) if idx not in incoherent_indices]
filtered_cell_indices = [idx for idx in cell_indices if idx not in incoherent_indices]

print("---------------")
print(f"Nombre d'index avant filtrage : {len(cell_indices)}")
print(f"Nombre d'index après filtrage : {len(filtered_cell_indices)}")
print("----------------")

# Créer un nouveau dictionnaire avec les données filtrées
filtered_data = {
    'feats': np.array(filtered_feats),
    'indices': np.array(filtered_cell_indices)  # Ajouter les indices filtrés
}

# Sauvegarder les données filtrées dans un nouveau fichier pkl
with open('true_filtered_cell_dataset.pkl', 'wb') as f:
    pickle.dump(filtered_data, f)

# Enregistrer les indices incohérents dans un fichier texte
with open('true_incoherent_indices.txt', 'w') as f:
    for idx in incoherent_indices:
        f.write(f"{idx}\n")

print(f"Le jeu de données filtré a été sauvegardé dans 'true_filtered_cell_dataset.pkl'.")
print(f"Les indices incohérents ont été sauvegardés dans 'true_incoherent_indices.txt'.")



---------------
Nombre d'index avant filtrage : 984007
Nombre d'index après filtrage : 984003
----------------
Le jeu de données filtré a été sauvegardé dans 'true_filtered_cell_dataset.pkl'.
Les indices incohérents ont été sauvegardés dans 'true_incoherent_indices.txt'.


In [20]:
# Charger les données filtrées
with open('true_filtered_cell_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
    features = data['feats']
    filtered_indices = set(data['indices'])  # Ensemble des indices filtrés

# Identifier les indices des cellules annotées
filtered_annotated_indices = set()

# S'assurer que les annotations sont uniques et cohérentes
for c in classes:
    for idx in annotations[c][0]:
        # Vérifier si l'indice est bien dans filtered_indices
        if idx in filtered_indices:
            filtered_annotated_indices.add(idx)

# Affichage des résultats
print(f"Nombre d'indices filtrés : {len(filtered_indices)}")
print(f"Nombre d'indices annotés : {len(filtered_annotated_indices)}")
print(f"Nombre d'indices non annotés : {len(filtered_indices - filtered_annotated_indices)}")

# Créer les indices des cellules non annotées
unlabeled_indices = list(filtered_indices - filtered_annotated_indices)
print(f"Nombre d'indices non annotés calculé : {len(unlabeled_indices)}")

Nombre d'indices filtrés : 984003
Nombre d'indices annotés : 987
Nombre d'indices non annotés : 983016
Nombre d'indices non annotés calculé : 983016


In [21]:
import numpy as np
import pickle

# Charger les features du jeu de données filtré
with open('true_filtered_cell_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
    features = data['feats']
    filtered_indices = data['indices']

# Initialiser les listes pour stocker les données formatées
X = []  # Features
y = []  # Labels
used_indices = []  # Pour garder une trace des indices utilisés

# Parcourir tous les indices filtrés
for idx in filtered_indices:
    if idx in combined_annotations:
        class_labels = combined_annotations[idx]
        # Déterminer la classe de la cellule
        positive_classes = [i for i, c in enumerate(classes) if class_labels.get(c, False)]

        if len(positive_classes) == 1:
            # Si la cellule appartient à une seule classe positive
            label = positive_classes[0]
        elif len(positive_classes) == 0:
            # Si la cellule n'appartient à aucune classe positive
            label = 3  # Classe pour "aucun des trois types"
        else:
            # Ne devrait pas arriver car on a filtré les incohérences
            continue

        # Trouver l'index dans le tableau features filtré
        feature_idx = np.where(filtered_indices == idx)[0][0]

        # Ajouter les features et le label
        X.append(features[feature_idx])
        y.append(label)
        used_indices.append(idx)

# Convertir en arrays numpy
X = np.array(X)
y = np.array(y)

print(f"Dimensions de X: {X.shape}")
print(f"Dimensions de y: {y.shape}")

# Afficher la distribution des classes
unique, counts = np.unique(y, return_counts=True)
print("\nDistribution des classes:")
for class_idx, count in zip(unique, counts):
    if class_idx < 3:
        print(f"Classe {class_idx} ({classes[class_idx]}): {count} échantillons")
    else:
        print(f"Classe {class_idx} (autre): {count} échantillons")

# Sauvegarder les données formatées
formatted_data = {
    'X': X,
    'y': y,
    'indices': used_indices,
    'class_names': classes + ('other',)
}

with open('true_formatted_cell_dataset.pkl', 'wb') as f:
    pickle.dump(formatted_data, f)

Dimensions de X: (987, 384)
Dimensions de y: (987,)

Distribution des classes:
Classe 0 (lymphocyte): 250 échantillons
Classe 1 (lymphoplasmocyte): 122 échantillons
Classe 2 (plasmocyte): 118 échantillons
Classe 3 (autre): 497 échantillons


In [24]:
import numpy as np
import pickle

# Charger les données filtrées
with open('true_filtered_cell_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
    features = data['feats']
    filtered_indices = set(data['indices'])  # Indices filtrés

# Calcul des indices annotés
annotated_indices = set()
for c in classes:
    for idx in annotations[c][0]:
        if idx in filtered_indices:  # Vérifier que l'indice fait bien partie du jeu de données filtré
            annotated_indices.add(idx)

# Vérifier le nombre d'indices annotés et non annotés
num_filtered = len(filtered_indices)
num_annotated = len(annotated_indices)
num_unlabeled = num_filtered - num_annotated

print(f"Nombre d'indices filtrés : {num_filtered}")
print(f"Nombre d'indices annotés : {num_annotated}")
print(f"Nombre d'indices non annotés calculé : {num_unlabeled}")

# Assurez-vous qu'il n'y a pas de doublons ou d'indices erronés
if num_unlabeled != 983016:
    print(f"Erreur : Le nombre de cellules non annotées ({num_unlabeled}) ne correspond pas à l'attendu (983016).")

# Extraire les indices non annotés
unlabeled_indices = list(filtered_indices - annotated_indices)  # Différence entre les indices filtrés et annotés

# Vérification du nombre d'indices non annotés
print(f"Nombre d'indices non annotés après calcul de la différence : {len(unlabeled_indices)}")

# S'assurer que les indices sont dans la bonne plage (0 à len(features)-1)
unlabeled_indices = [idx for idx in unlabeled_indices if idx < len(features)]

# Vérification que les indices sont valides
print(f"Nombre d'indices non annotés après validation : {len(unlabeled_indices)}")

# Créer le jeu de données non annoté
X_unlabeled = features[unlabeled_indices]  # Extraire les features des cellules non annotées

# Sauvegarder le jeu de données non annoté
unlabeled_data = {
    'X_unlabeled': X_unlabeled,
    'indices': unlabeled_indices
}

with open('unlabeled_cell_dataset.pkl', 'wb') as f:
    pickle.dump(unlabeled_data, f)

print(f"Le jeu de données non annoté a été sauvegardé dans 'unlabeled_cell_dataset.pkl'.")

Nombre d'indices filtrés : 984003
Nombre d'indices annotés : 987
Nombre d'indices non annotés calculé : 983016
Nombre d'indices non annotés après calcul de la différence : 983016
Nombre d'indices non annotés après validation : 983012
Le jeu de données non annoté a été sauvegardé dans 'unlabeled_cell_dataset.pkl'.


In [25]:
with open('unlabeled_cell_dataset.pkl', 'rb') as file:
    unlabeled_data = pickle.load(file)
print("Structure des données :", unlabeled_data.keys())

Structure des données : dict_keys(['X_unlabeled', 'indices'])


In [26]:
import numpy as np
import pickle

# Charger le jeu de données non annoté
with open('unlabeled_cell_dataset.pkl', 'rb') as f:
    unlabeled_data = pickle.load(f)
    X_unlabeled = unlabeled_data['X_unlabeled']
    unlabeled_indices = unlabeled_data['indices']

# Vérifier le nombre total de cellules non étiquetées disponibles
num_unlabeled_cells = len(X_unlabeled)
print(f"Nombre total de cellules non étiquetées disponibles : {num_unlabeled_cells}")

# Si le nombre de cellules non étiquetées est suffisant, en sélectionner 100 000
if num_unlabeled_cells >= 100000:
    # Sélectionner aléatoirement 100 000 indices
    selected_indices = np.random.choice(num_unlabeled_cells, 100000, replace=False)
    X_selected = X_unlabeled[selected_indices]  # Extraire les features correspondants aux indices sélectionnés
    selected_indices_list = list(np.array(unlabeled_indices)[selected_indices])  # Obtenir les indices sélectionnés

    # Sauvegarder le jeu de données réduit avec 100 000 exemples
    reduced_unlabeled_data = {
        'X_unlabeled': X_selected,
        'indices': selected_indices_list
    }

    with open('reduced_unlabeled_cell_dataset.pkl', 'wb') as f:
        pickle.dump(reduced_unlabeled_data, f)

    print("Le jeu de données réduit avec 100 000 cellules non étiquetées a été sauvegardé dans 'reduced_unlabeled_cell_dataset.pkl'.")
else:
    print(f"Erreur : Le nombre de cellules non étiquetées ({num_unlabeled_cells}) est insuffisant pour extraire 100 000 exemples.")

Nombre total de cellules non étiquetées disponibles : 983012
Le jeu de données réduit avec 100 000 cellules non étiquetées a été sauvegardé dans 'reduced_unlabeled_cell_dataset.pkl'.


## Training and evaluation 
Now we have our data ready!

Go ahead and implement a multi-class classifier. Don't forget to do train-test split or cross-validation to report performance. Good luck!

In [6]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Charger les données formatées
with open('true_formatted_cell_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
    # Extraire les features et les labels
    X = data['X']
    y = data['y']
    class_names = data['class_names']

# Séparer les données en ensembles d'entraînement et de test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialiser le classificateur K-Nearest Neighbors avec 5 voisins
k = 28 #int(math.sqrt(len(X_train)))
#print(f"Nombre de points d'entrainement : {len(X_train)}")
#print(f"Valeur de k calculée : {k}")

model = KNeighborsClassifier(n_neighbors=k, weights='uniform')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')

# Entraîner le modèle
print("Entraînement du modèle...")
model.fit(X_train_scaled, y_train)
print("Modèle entraîné avec succès.")

# Prédictions sur l'ensemble de test
y_pred_test = model.predict(X_test_scaled)

cm = confusion_matrix(y_test, y_pred_test)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'Matrice de confusion - KNN (k={k})')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.tight_layout()
plt.savefig(f'confusion_matrix_KNN(k={k}).png')
plt.close()

print("Taille de l'ensemble d'entraînement : ", X_train.shape[0])
print("Taille de l'ensemble de test : ", X_test.shape[0])
print("\nRésultats de la validation croisée : ")
print("-" * 50)
print(f"Accuracy moyenne : {scores.mean():.3f} (+/- {scores.std()*2:.3f}")
print("\nPerformances sur l'ensemble de test : ")
print("-" * 50)
print("\nRapport de classification détaillé : ")
print(classification_report(y_test, y_pred_test, target_names=class_names))

# Sauvegarder le modèle entraîné pour utilisation future
with open(f'knn_model(k={k}).pkl', 'wb') as f:
    pickle.dump(model, f)

print(f"Modèle sauvegardé dans 'knn_model(k={k}).pkl'.")


Entraînement du modèle...
Modèle entraîné avec succès.
Taille de l'ensemble d'entraînement :  789
Taille de l'ensemble de test :  198

Résultats de la validation croisée : 
--------------------------------------------------
Accuracy moyenne : 0.658 (+/- 0.024

Performances sur l'ensemble de test : 
--------------------------------------------------

Rapport de classification détaillé : 
                  precision    recall  f1-score   support

      lymphocyte       0.70      0.91      0.79        46
lymphoplasmocyte       0.38      1.00      0.55        15
      plasmocyte       0.52      0.92      0.67        25
           other       0.96      0.46      0.63       112

        accuracy                           0.67       198
       macro avg       0.64      0.82      0.66       198
    weighted avg       0.80      0.67      0.66       198

Modèle sauvegardé dans 'knn_model(k=28).pkl'.


In [12]:
# Nombre d'index testés (taille de y_test)
num_tested_indices = len(y_test)

# Nombre total d'index (taille de y)
num_total_indices = len(y)

# Afficher les résultats
print(f"Le modèle a testé {num_tested_indices} index.")
print(f"Le jeu de données contient un total de {num_total_indices} index.")


Le modèle a testé 198 index.
Le jeu de données contient un total de 987 index.


# Training and evaluation over IRIS

Now replicate the same multi class classification method but over the IRIS dataset.